In [ ]:
from nilearn import plotting,image
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
#import matplotlib as mpl
#mpl.rcParams['figure.dpi']= 200  

# Setup brainscapes

### Set a EBRAINS Knowledge Graph access token

Brainscapes retrieves some data from the EBRAINS Knowledge Graph, which requires
authentication. To do so, please follow these steps:

 1. If you do not yet have an EBRAINS account, register [here](https://ebrains.eu/register). As you are reading this notebook in the EBRAINS collaboratory, this is most probably not necessary at this point.
 2. Your EBRAINS account needs to be enabled for programmatic access to the EBRAINS Knowledge Graph to fetch metadata. This is formal step to acknowledge additional terms of use, and done quickly by emailing to the KG team. A link and template email to do so can be found right on top of the [Knowledge Graph developer page](https://kg.humanbrainproject.eu/develop.html).
 3. Create an authentication token for EBRAINS by visiting
[the EBRAINS authorization endpoint](https://nexus-iam.humanbrainproject.org/v0/oauth2/authorize). 
 4. Copy the token, and store it in the enviroment variable `HBP_AUTH_TOKEN` (just modify and execute the cell below accordingly).

Note that as of now, you have to get a new token (steps 3. and 4.) approximately every day.

In [ ]:
import webbrowser
from os import environ
webbrowser.open('https://nexus-iam.humanbrainproject.org/v0/oauth2/authorize')
token = input("Enter your token here: ")
environ['HBP_AUTH_TOKEN'] = token

Brainscapes maintains a local cache of retrieved data. It will automatically choose the system default of your user account on most common operating systems, but you an also choose an explicit folder by setting the environment variable `BRAINSCAPES_CACHEDIR`. We are not using this here.

In [ ]:
#!mkdir -p /tmp/brainscapescache
#environ['BRAINSCAPES_CACHEDIR'] = "/tmp/brainscapescache"

### Load the library

In [ ]:
import brainscapes as bs
bs.logger.setLevel("INFO") # show us some messages
#bs.clear_cache()

# Choosing a parcellation

We select the human atlas with cytoarchitectonic maps, and display maps in MNI152 and BigBrain space.

In [ ]:
atlas = bs.atlases.MULTILEVEL_HUMAN_ATLAS
atlas.select_parcellation(bs.parcellations.JULICH_BRAIN_PROBABILISTIC_CYTOARCHITECTONIC_MAPS_V2_5)

# in MNI 152 space
icbm_mri = atlas.get_template(bs.spaces.MNI_152_ICBM_2009C_NONLINEAR_ASYMMETRIC)
icbm_map = atlas.get_maps(bs.spaces.MNI_152_ICBM_2009C_NONLINEAR_ASYMMETRIC)
for m in image.iter_img(icbm_map):
    plotting.plot_stat_map(m)

# bigbrain
bigbrain_template = atlas.get_template(bs.spaces.BIG_BRAIN_HISTOLOGY,resolution=640)
bigbrain_map = atlas.get_maps(bs.spaces.BIG_BRAIN_HISTOLOGY,resolution=640)
for m in image.iter_img(bigbrain_map):
    plotting.plot_stat_map(m,bg_img=bigbrain_template)

# Finding and selection brain regions

Brainscapes allows to search for regions through an atlas object in a number of ways. You can use text fragments of region names, region identifiers, or label indices of the parcellation maps. The library does its best to resolve them for you. Let's start with a simple text string:

In [ ]:
atlas.select_region("v1")

As you see, V1 is subdivided into hemispheres. We can be more specific and select the left one. This is a leaf of the region hierarchy tree.

In [ ]:
atlas.select_region("v1 left")

In order to specifiy an element right away, we can use the 'regionnames' glossary stored in the atlas:

In [ ]:
atlas.select_region(atlas.regionnames.AREA_HOC1_V1_17_CALCS_LEFT_HEMISPHERE)

Of course, we can also search for regions, even across all known parcellations:

In [ ]:
matches = atlas.find('ventral',all_parcellations=True)
for m in matches:
    print("{}: {} ({})".format(m.parcellation,m.name,m.labelindex))

Regions which are labelleled in the parcellation map have a valid `labelindex` attached to them. We can also use this index to search and select regions:

In [ ]:
atlas.select_region(112)

Brainscapes supports access to continuous maps for some parcellations. For the Julich-Brain, this translates to the actual probability maps of each area. Let's look at a probability map of the frontal pole in the ICBM space.

In [ ]:
atlas.select_region('fp2 right')
pmap = atlas.selected_region.get_specific_map(bs.spaces.MNI_152_ICBM_2009C_NONLINEAR_ASYMMETRIC)
plotting.plot_roi(pmap)

# Extracting multimodal data features

Brainscapes provides access to data features of different modalities using the `get_features` method, which accepts a feature modality and is sensitive to the selections configured in the atlas (parcellation, region). If not particular selection is made, `get_features` considers all brain regions of the current parcellation.

### Extract transmitter receptor densities

Transmitter receptor density fingerprints are linked to brain regions by their name in the EBRAINS Knowledge Graph. Like any data feature, they are accessed using the `get_features` method of the atlas, which makes use of the current selection in the atlas. The `get_features` method knows from the specified data modality that the match is determined from the brain region identified. Receptor densities come as a nicely structured datatype. Amongst other things, they can visualize themselves in a plot.

If we don't specify a particular selection, the atlas will return all available receptor density features linked to the parcellation map:

In [ ]:
atlas.clear_selection()
features = atlas.get_features(
    bs.modalities.ReceptorDistribution)
print("Receptor density features found for the following regions:")
print("\n".join({f.region for f in features}))

If we select a region, the returned list is filtered accordingly. 

In [ ]:
atlas.select_region("v1")
features = atlas.get_features(
    bs.modalities.ReceptorDistribution)
for r in features:
    fig = r.plot(r.region)

### Extract spatial properties of brain regions

In [ ]:
props = atlas.regionprops(
    bs.spaces.MNI_152_ICBM_2009C_NONLINEAR_ASYMMETRIC)
for prop in props:
    print(prop)

### Extracting Gene Expressions from the Allen Atlas 

The atlas client can make calls to gene expression data from the Allen atlas and evaluate them in the ICBM space to find regional gene expression levels. It also has a list of available gene names for convenient selection. Gene expressions are linked to atlas regions by coordinates of their probes in MNI space. The `get_features` method detects this from the feature modality, and applies the mask of the regions that are currently selected in the atlas to filter the probes. We can visualize these filtered locations.

In [ ]:
features = atlas.get_features(
    bs.modalities.GeneExpression, 
    gene=bs.features.gene_names.GABARAPL2)
print(features[0])

# plot
all_coords = [tuple(g.location) for g in features]
mask = atlas.get_mask(bs.spaces.MNI_152_ICBM_2009C_NONLINEAR_ASYMMETRIC)
display = plotting.plot_roi(mask)
display.add_markers(all_coords,marker_size=5) 

### Retrieving a connectivity matrix
For brainscapes, a connectivity matrix is a data feature like the others below, and can be found using the same `get_features` function just by choosing another modality. Brainscapes knows from the modality type that this type of data does not match to the selected brain region in the atlas, but to the selected parcellation. Therefore, `get_features` return connectivity datasets that are defined for the selected parcellation. 

To learn about the nature of the provided connectivity, the `src_info` attribute provides a detailed description of the dataset. 

In [ ]:
# Get the first four connectivity matrices available for the parcellation
features = atlas.get_features(bs.modalities.ConnectivityMatrix)[:4]

# format dataset names for use as figure titles
from textwrap import wrap
titleformat = lambda text : "\n".join(wrap(text.replace('_',' '),20)) 

# plot the matrices
f,axs = plt.subplots(1,len(features))
for i,feature in enumerate(features):
    axs[i].imshow(feature.matrix,cmap=plt.cm.viridis)
    axs[i].set_title(titleformat(feature.src_name),size=8)
f.show()